In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import pandas as pd


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import cv2
from keras.models import model_from_json
import numpy as np

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/emot.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:

train_generator = train_data_gen.flow_from_directory(
    '/content/train',
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [ ]:
validation_generator = validation_data_gen.flow_from_directory(
		'/content/test',
		target_size=(48, 48),
		batch_size=64,
		color_mode="grayscale",
		class_mode='categorical')


Found 7178 images belonging to 7 classes.


In [ ]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',
						input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

emotion_model.summary()

cv2.ocl.setUseOpenCL(False)

initial_learning_rate = 0.0001
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=100000,
							decay_rate=0.96)

optimizer = Adam(learning_rate=lr_schedule)

emotion_model.compile(loss='categorical_crossentropy', optimizer=optimizer,
					metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 22, 22, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 128)       0         
 g2D)                                                   

In [ ]:
emotion_model.fit(
		train_generator,
		steps_per_epoch=28709 // 64,
		epochs=30,
		validation_data=validation_generator,
		validation_steps=7178 // 64)


Epoch 1/30
448/448 [==============================] - 24s 39ms/step - loss: 1.8079 - accuracy: 0.2527 - val_loss: 1.7487 - val_accuracy: 0.3117
Epoch 2/30
448/448 [==============================] - 13s 29ms/step - loss: 1.6499 - accuracy: 0.3554 - val_loss: 1.5660 - val_accuracy: 0.3965
Epoch 3/30
448/448 [==============================] - 14s 31ms/step - loss: 1.5421 - accuracy: 0.4095 - val_loss: 1.4715 - val_accuracy: 0.4385
Epoch 4/30
448/448 [==============================] - 14s 30ms/step - loss: 1.4655 - accuracy: 0.4375 - val_loss: 1.4014 - val_accuracy: 0.4714
Epoch 5/30
448/448 [==============================] - 13s 29ms/step - loss: 1.4044 - accuracy: 0.4644 - val_loss: 1.3709 - val_accuracy: 0.4778
Epoch 6/30
448/448 [==============================] - 13s 30ms/step - loss: 1.3487 - accuracy: 0.4898 - val_loss: 1.3257 - val_accuracy: 0.4986
Epoch 7/30
448/448 [==============================] - 13s 29ms/step - loss: 1.3038 - accuracy: 0.5068 - val_loss: 1.2906 - val_accuracy:

In [ ]:
emotion_model.save('dog_vs_cat_classification_model.h5')
print("Model saved successfully")

Model saved successfully


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
emotion_model.evaluate(validation_generator)


In [ ]:
import matplotlib.pyplot as plt

# Accuracy graph
plt.subplot(1, 2, 1)
plt.plot(accuracy, label='accuracy')
plt.plot(val_accuracy, label='val accuracy')
plt.title('Accuracy Graph')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# loss graph
plt.subplot(1, 2, 2)
plt.plot(loss, label='loss')
plt.plot(val_loss, label='val loss')
plt.title('Loss Graph')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful",
				3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}


In [ ]:
json_file = open('emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)


In [ ]:
cap = cv2.VideoCapture(0)
while True:
	# Find haar cascade to draw bounding box around face
	ret, frame = cap.read()
	frame = cv2.resize(frame, (1280, 720))
	if not ret:
		print(ret)
	# Create a face detector
	face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

	gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

	# detect faces available on camera
	num_faces = face_detector.detectMultiScale(gray_frame,
											scaleFactor=1.3, minNeighbors=5)

	# take each face available on the camera and Preprocess it
	for (x, y, w, h) in num_faces:
	cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
		roi_gray_frame = gray_frame[y:y + h, x:x + w]
		cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame,
															(48, 48)), -1), 0)

		# predict the emotions
		emotion_prediction = emotion_model.predict(cropped_img)
		maxindex = int(np.argmax(emotion_prediction))
		cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20),
					cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

	cv2.imshow('Emotion Detection', frame)
	if cv2.waitKey(1) & 0xFF == ord('q'):
		break

cap.release()
cv2.destroyAllWindows()
